In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Javascript
from modules import utility
import os
import pandas as pd
from medpy.metric.binary import hd, assd, dc, precision
import SimpleITK as sitk

def get_surface(img):
    a = sitk.GetImageFromArray(img)
    a = sitk.BinaryContour(a)
    a = sitk.GetArrayFromImage(a)
    return a.copy()

Using TensorFlow backend.


In [3]:
mhas = open('images.txt').readlines()
mhas = [i.replace('\n','') for i in mhas]

truths = open('truths.txt').readlines()
truths = [i.replace('\n','') for i in truths]

paths = open('paths.txt').readlines()
paths = [i.replace('\n','') for i in paths]

groups = open('groups.txt').readlines()
groups = [i.replace('\n','') for i in groups]

ct_test = open('ct_test.txt').readlines()
ct_test = [i.replace('\n','') for i in ct_test]

mr_test = open('mr_test.txt').readlines()
mr_test = [i.replace('\n','') for i in mr_test]

vtp_dir = '/home/marsdenlab/projects/DeepLofting/python/vtp/'
model_dir = '/home/marsdenlab/projects/DeepLofting/python/3d_models'

vtp_files = os.listdir(vtp_dir)

print ct_test
print vtp_files[:10]
IMAGES = ct_test+mr_test
print IMAGES
codes = ['i2ifc','i2i3d','i2i','drls','oof']
truth_files = [c for c in vtp_files if 'truth' in c]
print truth_files[:20]

test_images = []
for i in range(len(mhas)):
    if any([c in mhas[i] for c in IMAGES]):
        test_images.append((mhas[i],truths[i],paths[i],groups[i]))
        
#print test_images

['OSMSC0004', 'OSMSC0177', 'cabg11', 'OSMSC0179', 'OSMSC0144', 'OSMSC0148', 'OSMSC0150', 'OSMSC0119']
['0119_0001_i2ifc.LPA_br2_br1.vtp', '0150_0001_oof.celiac_hepatic.vtp', '0179_0001_truth.C_left.vtp', 'cabg11_all_oof.lca2.vtp', '0119_0001_i2i3d.LPA_br12.vtp', '0110_0001_i2i3d.right_iliac.vtp', '0177_0000_i2i3d.in_iliac_left.vtp', '0148_1001_i2i.renal_left.vtp', '0171_0001_i2i3d.R_incrd_b4.vtp', '0119_0001_i2i.RPA_br12_br4.vtp']
['OSMSC0004', 'OSMSC0177', 'cabg11', 'OSMSC0179', 'OSMSC0144', 'OSMSC0148', 'OSMSC0150', 'OSMSC0119', 'OSMSC0006', 'OSMSC0101', 'OSMSC0171', 'OSMSC0178', 'OSMSC0110']
['0179_0001_truth.C_left.vtp', '0179_0001_truth.R_ant.vtp', '0119_0001_truth.RPA_br8_br1.vtp', 'cabg11_all_truth.rsubcl3.vtp', '0119_0001_truth.RPA_br2.vtp', '0119_0001_truth.RPA_br9_br3.vtp', '0148_1001_truth.SMA.vtp', '0177_0000_truth.in_iliac_left.vtp', 'cabg11_all_truth.lsubcl3.vtp', '0171_0001_truth.L_incrd_b8.vtp', '0150_0001_truth.IMA.vtp', '0119_0001_truth.LPA_br4_br4.vtp', '0150_0001_tr

In [7]:
df = pd.read_csv('3derrs.csv')
df = df.drop(['Unnamed: 0'],axis=1)
print df.columns

Index([u'ASSD', u'FN', u'FP', u'TN', u'TP', u'code', u'hausdorf', u'image'], dtype='object')


In [12]:
strings = []
ct_test = [c.replace('OSMSC','') for c in ct_test]
mr_test = [c.replace('OSMSC','') for c in mr_test]
print mr_test
print ct_test
for CODE in ['CT','MR', 'ALL']:
    l = ct_test
    if CODE == 'MR': l = mr_test
    if CODE == "ALL": l = mr_test+ct_test
    strings = []
    f = open('{}_3d_err_.txt'.format(CODE),'w')
    for c in codes:

        d = df.loc[df['code']==c]
        t = [any([H==K for H in l]) for K in d['image']]

        d = d.loc[t]

        a = d.mean()
        
        s = "{} & ${:.3f}$$ \\".format(c,a['hausdorf'])
        
        strings.append((a['hausdorf'],s))

    strings = sorted(strings,key = lambda x: x[0])
    print CODE
    for s in strings:
        print s[1]
        f.write(s[1]+'\n')
    f.close()

['0006', '0101', '0171', '0178', '0110']
['0004', '0177', 'cabg11', '0179', '0144', '0148', '0150', '0119']
CT
i2ifc & $0.715$$ \
i2i & $0.742$$ \
oof & $0.754$$ \
drls & $0.762$$ \
i2i3d & $0.864$$ \
MR
i2ifc & $0.276$$ \
i2i & $0.301$$ \
oof & $0.387$$ \
i2i3d & $0.426$$ \
drls & $0.432$$ \
ALL
i2ifc & $0.584$$ \
i2i & $0.610$$ \
oof & $0.644$$ \
drls & $0.662$$ \
i2i3d & $0.730$$ \
